# Data

In [ ]:
import pandas as pd
import os

folder = 'Output'
paths = [i for i in os.listdir(folder) if '_s' in i and '_e' in i]# 'Data/DocListDF_GoogleSearch_s0_e500.p'

paths.sort()
L = []
for path in paths:
    try:
        df = pd.read_pickle(os.path.join(folder, path))
        print(path, '\t', len(df))
        
        L.append(len(df))
    except:
        print('Error', path)
    

# MDDO List

In [ ]:
mddo_path = 'Data/NPPESMDDOPhysician.p'
MDDO_NPI_included = pd.read_pickle(mddo_path)['NPI'].to_list()

# Get Links for One File

In [ ]:
import numpy as np

def get_website_fun(name):
    def get_website(data):
        results = [i for i in data if 'www.{}.com'.format(name) in i]
        if name == 'healthgrades':
            results = [i for i in results if 'physician' in i or 'provider' in i]
            results = [i for i in results if 'group-directory' not in i]
            
        elif name == 'zocdoc':
            results = [i for i in results if 'write-review' not in i]
            
        elif name == 'vitals':
            results = [i for i in results if 'doctors' in i and 'review' not in i]
        
        elif name == 'yelp':
            results = [i for i in results if 'search' not in i]
            
        # input
        L = []
        for i in results:
            if i not in L:
                L.append(i)
        results = L
        
        if len(results) == 0:
            return np.nan
        
        elif len(results) == 1:
            return results
        
        else:
            return results
        
    return get_website
 
    
websites2funcs = {name: get_website_fun(name) for name in ['healthgrades', 'zocdoc', 'vitals', 'ratemds', 'yelp']} 
websites2funcs
# get_healthgrade(data)

In [ ]:
folder = 'Output'
path = 'MissingOutputKeywords_GoogleSearch_s50000_e60000.p'

def get_PhysicianURL_Dict(path, folder = 'Output'):
    df = pd.read_pickle(os.path.join(folder, path))
    df = df[df['NPI'].isin(MDDO_NPI_included)].reset_index(drop = True)
    df = df.groupby('NPI').first().reset_index()

    D = {}

    for website, name_func in websites2funcs.items():
        df[website] = df['searched_urls'].apply(name_func)
        # df[-df[website].isna()]
        D[website] = df[-df[website].isna()][['NPI', website]]
    return df, D 


df, D = get_PhysicianURL_Dict(path, folder = 'Output')
s = {}
s['path'] = path
s['total'] = len(df)
for k, v in D.items(): 
    s[k] = len(v)
    
print(s)
df['searched_urls'].apply(lambda x: len(x)).sum()

# Loop All Google Files

In [ ]:
import pandas as pd
import os

folder = 'Output'
paths = [i for i in os.listdir(folder) if '_s' in i and '_e' in i]# 'Data/DocListDF_GoogleSearch_s0_e500.p'
# paths = ['MD_DocListDF_All_GoogleSearch_s0_e10000.p']

paths.sort()
L = []
PhysicianURL = []
for path in paths:
    df, D = get_PhysicianURL_Dict(path, folder = 'Output')
    s = {}
    s['path'] = path
    s['total'] = len(df)
    s['total_google_urls'] = df['searched_urls'].apply(lambda x: len(x)).sum()
    for k, v in D.items(): 
        s[k] = len(v)
    L.append(s)
    PhysicianURL.append(D)
    print(path, '\t', len(df))

In [ ]:
Description = pd.DataFrame(L)
d = {}
d['path'] = 'AllResult'
for k, v in Description.iloc[:, 1:].sum().to_dict().items(): d[k] = v
Description = pd.DataFrame([d] + L)
Description.to_csv('Output/URLReport.csv', index = False)
Description

# HealthGrades

In [ ]:
website = 'healthgrades'
df = pd.concat([D[website] for D in PhysicianURL]).reset_index(drop = True)
df = df.loc[df['NPI'].drop_duplicates().index]
NPI2HealthGrades = df
# NPI2HealthGrades.loc[NPI2HealthGrades['NPI'].drop_duplicates().index]
NPI2HealthGrades

In [ ]:
NPI2HealthGrades['NPI'].value_counts()

In [ ]:
NPI2HealthGrades['NPI'].nunique()

In [ ]:
folder = 'NPI2URL/healthgrades/'
path = os.path.join(folder, 'MD_Doc2GoogleURL_healthgrades.p')
WholeDF = NPI2HealthGrades
length = 100000
name = 'healthgrades'


initialNPI2URL = pd.read_pickle(path)
newdf = WholeDF[-WholeDF['NPI'].isin(initialNPI2URL['NPI'])].sort_values('NPI').reset_index(drop = True)
print(len(WholeDF))
print(len(initialNPI2URL))
print(len(newdf))

for i in range(1, int(len(newdf)/length) + 2):
    start = (i-1) * length
    end = i * length
    d = newdf.iloc[start:end, :].reset_index(drop = True)
    print(len(d))
    path = 'NPI2URL/{}/{}_mddo_v{}.p'.format(name, name, i)
    print(path)
    d.to_pickle(path)
    


# Vitals

In [ ]:
website = 'vitals'
NPI2Vitals = pd.concat([D[website] for D in PhysicianURL]).reset_index(drop = True)
df = NPI2Vitals
NPI2Vitals

In [ ]:

path = 'NPI2URL/vitals/MD_Doc2GoogleURL_vitals.p'
WholeDF = NPI2Vitals
length = 100000
name = 'vitals'


initialNPI2URL = pd.read_pickle(path)
newdf = WholeDF[-WholeDF['NPI'].isin(initialNPI2URL['NPI'])].sort_values('NPI').reset_index(drop = True)
print(len(WholeDF))
print(len(initialNPI2URL))
print(len(newdf))

for i in range(1, int(len(newdf)/length) + 2):
    start = (i-1) * length
    end = i * length
    d = newdf.iloc[start:end, :].reset_index(drop = True)
    print(len(d))
    path = 'NPI2URL/{}/{}_mddo_v{}.p'.format(name, name, i)
    print(path)
    d.to_pickle(path)
    


# ZocDoc

In [ ]:
website = 'zocdoc'
NPI2ZocDoc = pd.concat([D[website] for D in PhysicianURL]).reset_index(drop = True)
df = NPI2ZocDoc
NPI2ZocDoc

In [ ]:

path = 'NPI2URL/zocdoc/MD_Doc2GoogleURL_zocdoc.p'
WholeDF = NPI2ZocDoc
length = 100000
name = 'zocdoc'


initialNPI2URL = pd.read_pickle(path)
newdf = WholeDF[-WholeDF['NPI'].isin(initialNPI2URL['NPI'])].sort_values('NPI').reset_index(drop = True)
print(len(WholeDF))
print(len(initialNPI2URL))
print(len(newdf))

for i in range(1, int(len(newdf)/length) + 2):
    start = (i-1) * length
    end = i * length
    d = newdf.iloc[start:end, :].reset_index(drop = True)
    print(len(d))
    path = 'NPI2URL/{}/{}_mddo_v{}.p'.format(name, name, i)
    print(path)
    d.to_pickle(path)
    


# RateMDs

In [ ]:
website = 'ratemds'
NPI2RateMDs = pd.concat([D[website] for D in PhysicianURL]).reset_index(drop = True)
df = NPI2RateMDs
NPI2RateMDs

In [ ]:

path = 'NPI2URL/ratemds/MD_Doc2GoogleURL_ratemds.p'
WholeDF = NPI2RateMDs
length = 100000
name = 'ratemds'


initialNPI2URL = pd.read_pickle(path)
newdf = WholeDF[-WholeDF['NPI'].isin(initialNPI2URL['NPI'])].sort_values('NPI').reset_index(drop = True)
print(len(WholeDF))
print(len(initialNPI2URL))
print(len(newdf))

for i in range(1, int(len(newdf)/length) + 2):
    start = (i-1) * length
    end = i * length
    d = newdf.iloc[start:end, :].reset_index(drop = True)
    print(len(d))
    path = 'NPI2URL/{}/{}_mddo_v{}.p'.format(name, name, i)
    print(path)
    d.to_pickle(path)
    


# Yelp

In [ ]:
website = 'yelp'
NPI2Yelp = pd.concat([D[website] for D in PhysicianURL]).reset_index(drop = True)
df = NPI2Yelp
NPI2Yelp

In [ ]:
path = 'NPI2URL/yelp/MD_Doc2GoogleURL_yelp.p'
WholeDF = NPI2Yelp
length = 100000
name = 'yelp'


initialNPI2URL = pd.read_pickle(path)
newdf = WholeDF[-WholeDF['NPI'].isin(initialNPI2URL['NPI'])].sort_values('NPI').reset_index(drop = True)
print(len(WholeDF))
print(len(initialNPI2URL))
print(len(newdf))

for i in range(1, int(len(newdf)/length) + 2):
    start = (i-1) * length
    end = i * length
    d = newdf.iloc[start:end, :].reset_index(drop = True)
    print(len(d))
    path = 'NPI2URL/{}/{}_mddo_v{}.p'.format(name, name, i)
    print(path)
    d.to_pickle(path)